### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [1]:
import json
import re
from collections import Counter, namedtuple

### 載入資料

In [2]:
with open('./WebNews.json', 'r') as f:
    news_data = json.load(f)

In [ ]:
news_data[:2]

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [ ]:
#取出新聞內文
corpus_list = [content['detailcontent'] for content in news_data]

#去除HTML Tags與標點符號(只保留英文、數字、中文)
corpus_list =  [re.sub('<[^<]+?>', '', corpus) for corpus in corpus_list]
corpus_list[0]

#print(len(corpus_list))
N = 3
test = [tuple(corpus_list[0][i:i+N]) for i in range(len(corpus_list[0])-N+1)]
print(test)
print(Counter(test))

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [40]:
def ngram(documents, N=2):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率

    for doc in documents:
        # 在每個文章錢加上起始(<start>)與結束符號(<end>)
        split_words = f"{doc}"
        # 計算分子
        [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
        # 計算分母
        [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]
    
    #計算分子詞頻
    total_word_counter = Counter(total_grams)
    #計算分母詞頻
    word_counter = Counter(words)
    
    
    #計算所有N-gram預測字詞的機率
    for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
            
        ngram_prediction[word].add(Word(key[-1], total_word_counter[key]/word_counter[key[:N-1]]))
        
    return ngram_prediction

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [44]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(corpus_list, 3)
for word, pred in four_gram_pred.items():
    ### <your code> ###
    sorted(pred, key=lambda x: x.prob, reverse=True)

In [ ]:
# 執行時因為字典數龐大，顯示會較慢
four_gram_pred

In [46]:
#給定字詞，使用ngram預測下一個字的機率(顯示top 10)
text = '市長'
next_words = list(four_gram_pred[text])[:10]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 帶, probability: 0.0004166666666666667
next word: 肖, probability: 0.00020833333333333335
next word: 自, probability: 0.00020833333333333335
next word: 多, probability: 0.0004166666666666667
next word: 主, probability: 0.0004166666666666667
next word: 將, probability: 0.0008333333333333334
next word: 已, probability: 0.0004166666666666667
next word: :, probability: 0.00020833333333333335
next word: 除, probability: 0.0014583333333333334
next word: 更, probability: 0.0004166666666666667


可自行嘗試使用不同的N搭建不同的N-Gram模型的效果